# 1. Module Importing 

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

C:\Users\lsj32\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\lsj32\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


# 2. Hyperparameter 

In [24]:
batch_size = 100 #원래 256.. 일단 100으로 해보자.
learning_rate = 0.001 #원래 0.0002로 설정되어 있었음...
num_epoch = 20

# 3. Data 

정규화시 평균인 0.1307과 표준편차 0.3081이 어떻게 구해진 건지 궁금하다면 아래의 링크를 참고하세요.  
http://www.gisdeveloper.co.kr/?p=8168

In [25]:
# 입력 데이터 정규화는 transform을 통해 가능합니다.
# 여기서 mean, std는 미리 계산된 값입니다.
# 새 데이터를 사용할 때, 직접 구하셔야 합니다. 
# mean, std가  각각 1개인 이유는 MNIST 데이터의 채널이 하나이기 때문입니다. 
# 채널이 세 개인 color 이미지를 다루실 때에는 채널 당 하나씩, 총 세 개의 mean과 std가 들어갑니다. 

mnist_train = dset.MNIST("./", train=True, 
                         transform=transforms.Compose([
                             transforms.Resize(34),                             # 원래 28x28인 이미지를 34x34로 늘립니다.
                             transforms.CenterCrop(28),                         # 중앙 28x28만을 뽑아냅니다.
                             #transforms.RandomHorizontalFlip(),                 # 랜덤하게 좌우반전 합니다.
                             #transforms.Lambda(lambda x: x.rotate(90)),         # 람다함수를 이용해 90도 회전해줍니다.
                             transforms.ToTensor(),                             # 이미지를 텐서로 변형합니다.(데이터를 0에서 255까지 있는 값을 0에서 1사이 값으로 변환)
                             transforms.Normalize(mean=(0.1307,), std=(0.3081,)) # 정규화입니다. (평균/255=0.1307,표준편차/255=0.3081)
                         ]),
                         target_transform=None, 
                         download=True)
mnist_test = dset.MNIST("./", train=False, 
                        transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean=(0.1307,), std=(0.3081,))
                        ]),
                        target_transform=None, 
                        download=True)

In [26]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
mnist_test.__getitem__(0)[0].size(), mnist_test.__len__()

torch.Size([1, 28, 28]) 60000


(torch.Size([1, 28, 28]), 10000)

In [27]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=0,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=0,drop_last=True)

# 4. Model  

In [32]:
# Batch Normalization 
# 입력 데이터를 정규화하는것처럼 연산을 통과한 결과값을 정규화할 수 있습니다.
# 그 다양한 방법중에 대표적인것이 바로 Batch Normalization이고 이는 컨볼루션 연산처럼 모델에 한 층으로 구현할 수 있습니다.
# https://pytorch.org/docs/stable/nn.html?highlight=batchnorm#torch.nn.BatchNorm2d
# nn.BatchNorm2d(x)에서 x는 입력으로 들어오는 채널의 개수입니다.

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,3,padding=1),  # 28 x 28
            nn.BatchNorm2d(16),
            nn.ReLU(),
            #nn.Dropout2d(0.2),
            nn.Conv2d(16,32,3,padding=1), # 28 x 28
            nn.BatchNorm2d(32),
            nn.ReLU(),
            #nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2),            # 14 x 14
            nn.Conv2d(32,64,3,padding=1), # 14 x 14
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2)             #  7 x 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7,100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100,10)
        )       

      #Initialization. 
      # 초기화 하는 방법
        # 모델의 모듈을 차례대로 불러옵니다.

        for m in self.modules():
            # 만약 그 모듈이 nn.Conv2d인 경우
            if isinstance(m, nn.Conv2d):
                '''
                # 작은 숫자로 초기화하는 방법
                # 가중치를 평균 0, 편차 0.02로 초기화합니다.
                # 편차를 0으로 초기화합니다.
                m.weight.data.normal_(0.0, 0.02)
                m.bias.data.fill_(0)
                
                # Xavier Initialization
                # 모듈의 가중치를 xavier normal로 초기화합니다.
                # 편차를 0으로 초기화합니다.
                init.xavier_normal(m.weight.data)
                m.bias.data.fill_(0)
                '''
                
                # Kaming Initialization
                # 모듈의 가중치를 kaming he normal로 초기화합니다.
                # 편차를 0으로 초기화합니다.
                init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)
            
            # 만약 그 모듈이 nn.Linear인 경우
            elif isinstance(m, nn.Linear):
                '''
                # 작은 숫자로 초기화하는 방법
                # 가중치를 평균 0, 편차 0.02로 초기화합니다.
                # 편차를 0으로 초기화합니다.
                m.weight.data.normal_(0.0, 0.02)
                m.bias.data.fill_(0)
                
                # Xavier Initialization
                # 모듈의 가중치를 xavier normal로 초기화합니다.
                # 편차를 0으로 초기화합니다.
                init.xavier_normal(m.weight.data)
                m.bias.data.fill_(0)
                '''
                
                # Kaming Initialization
                # 모듈의 가중치를 kaming he normal로 초기화합니다.
                # 편차를 0으로 초기화합니다.
                init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)

        






    def forward(self,x):
        out = self.layer(x)
        out = out.view(-1,64*7*7)
        out = self.fc_layer(out)
        return out

# 5. Loss & Optimizer 

In [33]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #weight_decay=0.01, L2정규화의 lambda=weight_decay


cuda:0


# 6.Train


In [8]:
# L1,L2정규화 직접 정의해보기 : 실제 사용할 때는 L2정규화는 optimizer의 weight_decay사용하면 되고, L2 정규화는 train시 for문 안에 넣어줘야 합니다. 

#all_parameters = torch.cat([x.view(-1) for x in model.parameters()]) 
# #특정 부분에 제약을 걸고싶으면 
#all_parameters = torch.cat([x.view(-1) for x in model.layer.parameters()])  이렇게 바꿔주시면 됩니다. 
lambda1 = 0.05
#l1_regularization = lambda1 * torch.norm(all_parameters, 1)
#l2_regularization = lambda1 * torch.norm(all_parameters, 2)

#loss = loss_func + l1_regularization 
#loss = loss_func + l2_regularization 

6-1 optimizer 정의 시 weight_decay=0(default)로 두면 L1,L2정규화 사용 안하고 train ,  
weight_decay에 0초과의 값을 지정하면 L2정규화하여 train

for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad() 
        #만약 optimzer에서 weight_decay를 default는 0이고 0보다 큰 값으로 주면 L2 정규화를 해준 것
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        print(loss)        

6-2 L1 정규화하여 train 

In [34]:
# pytorch는 L1정규화는 지원하지 않으므로 직접 정의하여 사용해야한다.
lambda1 = 0.05
model.train()
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        all_parameters = torch.cat([x.view(-1) for x in model.parameters()]) 
        l1_regularization = lambda1 * torch.norm(all_parameters, 1)
        loss = loss_func(output,y_)+ l1_regularization 
        loss.backward()
        optimizer.step()
        
    print(loss) 

tensor(117.3044, device='cuda:0', grad_fn=<AddBackward0>)
tensor(51.2362, device='cuda:0', grad_fn=<AddBackward0>)
tensor(34.1821, device='cuda:0', grad_fn=<AddBackward0>)
tensor(24.4600, device='cuda:0', grad_fn=<AddBackward0>)
tensor(19.0403, device='cuda:0', grad_fn=<AddBackward0>)
tensor(15.6630, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13.9084, device='cuda:0', grad_fn=<AddBackward0>)
tensor(12.7306, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.8553, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11.4455, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.7796, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10.4687, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.9922, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.7202, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.3322, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.0206, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8.6683, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8.3359, device='cuda:0', gra

7.Test

In [35]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
  for image,label in test_loader:
      x = image.to(device)
      y_= label.to(device)

      output = model.forward(x)
      _,output_index = torch.max(output,1)

      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 89.80999755859375
